# Fix mistakes and make training patches

This notebook shows how to fix mistakes on the labels and make new training patches.

First load the image and previous segmentrations

In [22]:
from skimage.io import imread, imsave
import napari
from tnia.deeplearning.dl_helper import make_patch_directory
import os
from pathlib import Path
import json

tnia_images_path = Path(r'D:/images')
parent_path=Path(tnia_images_path / r'tnia-python-images/imagesc/2024_01_10_insect_egg_count')

im_rgb = imread(os.path.join(parent_path / "I1_F1_O20_SWI_R1_13-09-23.tif"))
labels = imread(os.path.join(parent_path, "I1_F1_O20_SWI_R1_13-09-23_cellpose_labels.tif"))


## edit in napari

Now edit the labels in Napari.  When I did it, I just fixed some obvious mixes and tweaked some of the clusters a bit (the clusters are really hard so who knows if I imporved the situation there, I did fix some obvious misses though)

In [23]:
viewer = napari.Viewer()
viewer.add_image(im_rgb, name='im_rgb')
viewer.add_labels(labels, name='labels')

<Labels layer 'labels' at 0x2700142a430>

## Save modified labels

In [11]:

imsave(os.path.join(parent_path / "I1_F1_O20_SWI_R1_13-09-23_corrected_labels2.tif"), labels)

C:\Users\bnort\AppData\Local\Temp\ipykernel_24500\2120156986.py:1: UserWarning: D:\images\tnia-python-images\imagesc\2024_01_10_insect_egg_count\I1_F1_O20_SWI_R1_13-09-23_corrected_labels.tif is a low contrast image
  imsave(os.path.join(parent_path / "I1_F1_O20_SWI_R1_13-09-23_corrected_labels.tif"), labels)


## Create folders for input and ground truth patches

When we train a neural network we usually use small patches.  So create a directory for input (the original image) and ground truth (the labels) patches, then save the patches.

In [5]:
train_path= parent_path / 'labels'
image_patch_path =  train_path / 'input0'
label_patch_path =  train_path / 'ground truth0'

if not os.path.exists(train_path):
    os.makedirs(train_path)

if not os.path.exists(image_patch_path):
    os.makedirs(image_patch_path)

if not os.path.exists(label_patch_path):
    os.makedirs(label_patch_path)

patch_base_name = 'eggs'
axes = 'YXC'
sub_sample = 1

make_patch_directory(1, 1, train_path)

# Load the existing JSON data which is created when making the patch directory and append addition information to it
json_file = train_path / "info.json"

with open(json_file, 'r') as infile:
    data = json.load(infile)

# add the sub_sample information to the JSON file
data['sub_sample'] = sub_sample 
data['axes'] = axes

# Write the modified data back to the JSON file
with open(json_file, 'w') as outfile:
    json.dump(data, outfile)


In [8]:
from skimage.io import imsave

patch_size = (256, 256)


for i in range(0, im_rgb.shape[0], patch_size[0]):
    for j in range(0, im_rgb.shape[1], patch_size[1]):
        im_patch = im_rgb[i:i+patch_size[0], j:j+patch_size[1], :]
        label_patch = labels[i:i+patch_size[0], j:j+patch_size[1]]
        im_patch_name = os.path.join(image_patch_path, patch_base_name+f"_{i}_{j}.tif")
        label_patch_name = os.path.join(label_patch_path, patch_base_name+f"_{i}_{j}.tif")
        
        print(im_patch_name)
        print(label_patch_name)
        print(im_patch.shape)
        print(label_patch.shape)
        print('-----------------')
        imsave(im_patch_name, im_patch)
        imsave(label_patch_name, label_patch)
        

D:\images\tnia-python-images\imagesc\2024_01_10_insect_egg_count\labels\input0\eggs_0_0.tif
D:\images\tnia-python-images\imagesc\2024_01_10_insect_egg_count\labels\ground truth0\eggs_0_0.tif
(256, 256, 3)
(256, 256)
-----------------
D:\images\tnia-python-images\imagesc\2024_01_10_insect_egg_count\labels\input0\eggs_0_256.tif
D:\images\tnia-python-images\imagesc\2024_01_10_insect_egg_count\labels\ground truth0\eggs_0_256.tif
(256, 256, 3)
(256, 256)
-----------------
D:\images\tnia-python-images\imagesc\2024_01_10_insect_egg_count\labels\input0\eggs_0_512.tif
D:\images\tnia-python-images\imagesc\2024_01_10_insect_egg_count\labels\ground truth0\eggs_0_512.tif
(256, 256, 3)
(256, 256)
-----------------
D:\images\tnia-python-images\imagesc\2024_01_10_insect_egg_count\labels\input0\eggs_0_768.tif
D:\images\tnia-python-images\imagesc\2024_01_10_insect_egg_count\labels\ground truth0\eggs_0_768.tif
(256, 256, 3)
(256, 256)
-----------------
D:\images\tnia-python-images\imagesc\2024_01_10_inse

C:\Users\bnort\AppData\Local\Temp\ipykernel_24500\1970234879.py:19: UserWarning: D:\images\tnia-python-images\imagesc\2024_01_10_insect_egg_count\labels\ground truth0\eggs_0_0.tif is a low contrast image
  imsave(label_patch_name, label_patch)
C:\Users\bnort\AppData\Local\Temp\ipykernel_24500\1970234879.py:19: UserWarning: D:\images\tnia-python-images\imagesc\2024_01_10_insect_egg_count\labels\ground truth0\eggs_0_256.tif is a low contrast image
  imsave(label_patch_name, label_patch)
C:\Users\bnort\AppData\Local\Temp\ipykernel_24500\1970234879.py:19: UserWarning: D:\images\tnia-python-images\imagesc\2024_01_10_insect_egg_count\labels\ground truth0\eggs_0_512.tif is a low contrast image
  imsave(label_patch_name, label_patch)
C:\Users\bnort\AppData\Local\Temp\ipykernel_24500\1970234879.py:19: UserWarning: D:\images\tnia-python-images\imagesc\2024_01_10_insect_egg_count\labels\ground truth0\eggs_0_768.tif is a low contrast image
  imsave(label_patch_name, label_patch)
C:\Users\bnort\App